In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bioni

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-16 03:58:20--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.99MB/s    in 0.2s    

2021-08-16 03:58:20 (5.99 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url_us_total ="https://ucb-data-group-project-zillow-s3.s3.us-west-1.amazonaws.com/us_total_final.csv"
spark.sparkContext.addFile(url_us_total)
us_total_df = spark.read.csv(SparkFiles.get("us_total_final.csv"), sep=",", header=True, inferSchema=True)

In [5]:
# Show DataFrame
us_total_df.show()

+--------------------+----------+---------+-------------+----------+------------+----------+----------+--------------------+---------------+
|         region_date|      date|size_rank|  region_name|  latitude|   longitude|list_price|sale_price|list_divided_by_sale|list_minus_sale|
+--------------------+----------+---------+-------------+----------+------------+----------+----------+--------------------+---------------+
|unitedstates20171104|2017-11-04|        0|United States|39.7837304|-100.4458825|  275448.0|  235099.0|   85.35150010165258|        40349.0|
|unitedstates20171111|2017-11-11|        0|United States|39.7837304|-100.4458825|  275448.0|  234750.0|   85.22479742092881|        40698.0|
|unitedstates20171118|2017-11-18|        0|United States|39.7837304|-100.4458825|  275448.0|  235750.0|    85.5878423513694|        39698.0|
|unitedstates20171125|2017-11-25|        0|United States|39.7837304|-100.4458825|  274975.0|  235916.0|   85.79543594872261|        39059.0|
|unitedstates

In [6]:
url_ca_state ="https://ucb-data-group-project-zillow-s3.s3.us-west-1.amazonaws.com/ca_state_final.csv"
spark.sparkContext.addFile(url_ca_state)
ca_state_df = spark.read.csv(SparkFiles.get("ca_state_final.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
ca_state_df.show()

+------------------+----------+---------+-----------+----------+-----------+----------+----------+--------------------+------------------+
|       region_date|      date|size_rank|region_name|  latitude|  longitude|list_price|sale_price|list_divided_by_sale|   list_minus_sale|
+------------------+----------+---------+-----------+----------+-----------+----------+----------+--------------------+------------------+
|california20171104|2017-11-04|     36.4| California|36.7014631|-118.755997|  614123.7|  508281.3|   82.76529630756801|105842.39999999997|
|california20171111|2017-11-11|     36.4| California|36.7014631|-118.755997|  613946.7|  514450.0|   83.79391891836865| 99496.69999999995|
|california20171118|2017-11-18|     36.4| California|36.7014631|-118.755997|  614898.4|  516943.9|   84.06980730475149|           97954.5|
|california20171125|2017-11-25|     36.4| California|36.7014631|-118.755997|  612118.7|  517912.7|    84.6098477305137| 94205.99999999994|
|california20171202|2017-12

In [7]:
url_ca_city ="https://ucb-data-group-project-zillow-s3.s3.us-west-1.amazonaws.com/california_city_final.csv"
spark.sparkContext.addFile(url_ca_city)
ca_city_df = spark.read.csv(SparkFiles.get("california_city_final.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
ca_city_df.show()

+--------------------+----------+---------+--------------------+----------+------------+----------+----------+--------------------+---------------+
|         region_date|      date|size_rank|         region_name|  latitude|   longitude|list_price|sale_price|list_divided_by_sale|list_minus_sale|
+--------------------+----------+---------+--------------------+----------+------------+----------+----------+--------------------+---------------+
|    sandiego20171104|2017-11-04|       17|       San Diego, CA|32.7174202|-117.1627728|  728005.0|  541438.0|   74.37284084587331|       186567.0|
|sanfrancisco20171104|2017-11-04|       11|   San Francisco, CA|37.7790262| -122.419906|  867930.0|  781063.0|   89.99147396679456|        86867.0|
|   riverside20171104|2017-11-04|       13|       Riverside, CA|33.9533546|-117.3961623|  388757.0|  342999.0|   88.22966531792353|        45758.0|
|losangeles-longbe...|2017-11-04|        2|Los Angeles-Long ...| 33.782237|-118.1519248|  812252.0|  611250.0|  

In [8]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://ucb-data-group-project-zillow.csaw135fqqkl.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [9]:
# Write DataFrame to zillow_final table in RDS
us_total_df.write.jdbc(url=jdbc_url, table='zillow_final', mode=mode, properties=config)

In [10]:
# Write DataFrame to zillow_final table in RDS
ca_state_df.write.jdbc(url=jdbc_url, table='zillow_final', mode=mode, properties=config)

In [11]:
# Write DataFrame to zillow_final table in RDS
ca_city_df.write.jdbc(url=jdbc_url, table='zillow_final', mode=mode, properties=config)